# Прием и обработка твитов микробатчем

## Инициализация

In [8]:
import org.apache.spark.sql.types.{StructType, StringType, IntegerType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.toree.kernel.api
import java.util.Calendar

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


## Чтение модели

In [3]:
val modelPath = "/home/jovyan/models/spark-ml-model"
val model = PipelineModel.load(modelPath)

modelPath = /home/jovyan/models/spark-ml-model
model = pipeline_e3869da6fdc9


pipeline_e3869da6fdc9

## Определяем схему и инициируем потоковый датафрейм

In [25]:
val inputStreamPath = "/home/jovyan/work/events-stream"
//val outputStreamPath = "/home/jovyan/work/events-stream-out"

// Определяем udf для получения probability по 0 и 1
val getProbability =
    udf(
        (prediction: org.apache.spark.ml.linalg.Vector, pos: Integer) =>
        {
            prediction(pos)
        }
    )

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("hiddentargetclue", IntegerType)
    .add("timestamp", TimestampType)

inputStreamPath = /home/jovyan/work/events-stream
getProbability = UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))
dataSchema = StructType(StructField(tweet,StringType,true), StructField(hiddentargetclue,IntegerType,true), StructField(timestamp,TimestampType,true))


StructType(StructField(tweet,StringType,true), StructField(hiddentargetclue,IntegerType,true), StructField(timestamp,TimestampType,true))

In [61]:
/*
                    .select(
                        $"arrived_key",
                        $"tweet",
                        // $"hiddentargetclue",
                        (getProbability($"probability",lit(0))).alias("Negative Probability")
                    )
*/

/*
                model.transform(batchDF)
                    .select(
                        $"timestamp",
                        $"tweets"
                    )
*/

val inputDF = spark
    .readStream
    .schema(dataSchema)
//    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)
    .withWatermark("timestamp", "15 seconds")
    .groupBy(
        window($"timestamp", "10 seconds", "5 seconds").alias("timestamp")
    )
    .count()
    //.agg(count("*")).alias("tweets")

//val wStream = inputDF.writeStream
//    .option("checkpointLocation", outputStreamPath)
//.start(outputStreamPath)

inputDF = [timestamp: struct<start: timestamp, end: timestamp>, count: bigint]


lastException: Throwable = null


[timestamp: struct<start: timestamp, end: timestamp>, count: bigint]

## Микробатч приема твитов

In [62]:
var fRuns = 0
var fTweets:Long = 0

// Микробатч для вывода результата предсказания
// Выводится вероятность негативного твита
// В задании написано, что это последняя колонка, но она здесь вроде первая (в позиции 0)
val stream = inputDF.writeStream.foreachBatch {
    (batchDF: DataFrame, batchId: Long) => {
        try {
            fRuns += 1
            //batchDF.foreach {
            //    row:Row => {
            //        fTweets += 1
            //        //row.getAs("tweets")
            //    }
            //}
            batchDF.show()
            //print(s"${Calendar.getInstance().toInstant} - loaded $fTweets rows from the events stream $fRuns times"+13.toChar)
            // Применяем модель и получаем соотв. датасет с предсказаниями
//            globDF = 
//                model.transform(batchDF)
//                    .select(
//                        $"timestamp",
//                        $"tweets"
//                    )
            
        } catch {
            case e:Throwable => {
                println(e.getMessage)
            }
        }
    }
}.start()

fRuns = 0
fTweets = 0
stream = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5982365e


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5982365e

+---------+-----+
|timestamp|count|
+---------+-----+
+---------+-----+

+--------------------+-----+
|           timestamp|count|
+--------------------+-----+
|[2020-01-27 23:48...|   32|
|[2020-01-27 23:56...|   17|
|[2020-01-27 23:58...|   19|
|[2020-01-27 23:49...|   38|
|[2020-01-27 23:53...|   27|
|[2020-01-27 23:43...|   32|
|[2020-01-27 23:47...|   39|
|[2020-01-27 23:54...|   17|
|[2020-01-27 23:57...|    9|
|[2020-01-27 23:37...|   37|
|[2020-01-27 23:35...|   28|
|[2020-01-27 23:50...|   16|
|[2020-01-28 00:00...|   36|
|[2020-01-28 00:00...|   20|
|[2020-01-28 00:03...|   16|
|[2020-01-27 23:46...|   21|
|[2020-01-28 00:04...|   20|
|[2020-01-28 00:06...|   13|
|[2020-01-27 23:50...|   22|
|[2020-01-27 23:34...|   21|
+--------------------+-----+
only showing top 20 rows

Job aborted due to stage failure: Task 6 in stage 799.0 failed 1 times, most recent failure: Lost task 6.0 in stage 799.0 (TID 21848, localhost, executor driver): java.lang.IllegalStateException: Error rea

## Останов чтения потока

In [63]:
stream.stop()